In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [3]:
# get the data
from datasets import load_dataset
data = load_dataset('ola13/small-the_pile-dedup')

In [4]:
len(data['train']['text'])

100000

In [5]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/workspace/cache/' # change or comment out as desired 
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_name_or_path, revision, device):
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path, device_map=device, revision=revision, trust_remote_code=False)
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True, padding_side="left")
    tokenizer.pad_token_id = 0
    return model, tokenizer

device = 'cuda:0'
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
revision = 'gptq-4bit-32g-actorder_True'
user_tag = "[INST] "
assistant_tag = " [/INST]"

model, tokenizer = load_model(model_name_or_path, revision, device)

model_mistral=model
tokenizer_mistral=tokenizer

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [7]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer

model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/pythia-70m-deduped")

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m-deduped")
tokenizer.pad_token_id = 0

tokenizer_pythia = tokenizer
model_pythia = model
model_pythia.to('cuda:0')

config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:

device = 'cuda:0'
model_name_or_path = 'TheBloke/TinyLlama-1.1B-Chat-v1.0-GPTQ'
revision = 'gptq-4bit-32g-actorder_True'

model, tokenizer = load_model(model_name_or_path, revision, device)

model_tiny=model
tokenizer_tiny=tokenizer

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/825M [00:00<?, ?B/s]

The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [10]:
revision = 'gptq-4bit-32g-actorder_True'
model_name_or_path = 'TheBloke/Mistral-7B-v0.1-GPTQ'

model, tokenizer = load_model(model_name_or_path, revision, device)

model_mistralpre = model
tokenizer_mistralpre=tokenizer

config.json:   0%|          | 0.00/962 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/962 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [11]:
# get activations

device = 'cuda:0'
# model = model_pythia.to(device)
# model = model_mistral.to(device)
# model = model_tiny.to(device)
# tokenizer = tokenizer_tiny
model = model_mistralpre
tokenizer = tokenizer_mistralpre


bs = 16
layer = 3
context_len = 128
all_acts = []
for i in tqdm(range(1024, 2048, bs)):
    text = data['train']['text'][i:i+bs]
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    inputs_mod = {'input_ids': inputs['input_ids'][:, :context_len], 'attention_mask': inputs['attention_mask'][:, :context_len]}
    output = model(**inputs_mod, output_hidden_states=True)
    acts = output['hidden_states'][layer+1].detach().reshape(bs*context_len, -1)
    all_acts.extend(acts)
all_acts = torch.stack(all_acts)

100%|██████████| 64/64 [01:01<00:00,  1.05it/s]
